# 大A 股票数据预处理

本 Notebook 用于从 Tushare 拉取 XXXXXX（e.g. 000155）的**全部历史日线数据**和 **daily_basic 数据**，并保存到本地文件。

## 1. 导入依赖包

```bash
pip install tushare pandas
```

In [23]:
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import time
import os

print(f"tushare version: {ts.__version__}")
print(f"pandas version: {pd.__version__}")

tushare version: 1.4.24
pandas version: 2.2.2


## 2. 配置 Tushare Token

**请替换为你的 Tushare Pro Token**

获取方式：https://tushare.pro/register

In [24]:
# ⚠️ 请替换为你的 Token
TOKEN = 'dfb2e1674d624924723b9d625162d1cef97f05b6275c2344034fb07f'

# 初始化 Tushare Pro API
pro = ts.pro_api(TOKEN)



## 3. 定义数据获取函数

In [25]:
def get_all_daily_data(ts_code, start_date, sleep_time=0.3):
    """
    分批获取全部历史日线数据
    
    Parameters:
    -----------
    ts_code : str
        股票代码，如 '000155.SZ'
    start_date : str
        开始日期，格式 'YYYYMMDD'
    sleep_time : float
        每次请求间隔时间（秒），避免请求过快
    
    Returns:
    --------
    pd.DataFrame
        包含全部历史日线数据的 DataFrame
    """
    all_data = []
    
    # 计算日期范围
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.now()
    
    current_start = start
    
    print(f"开始获取日线数据: {start_date} ~ {end.strftime('%Y%m%d')}")
    print("=" * 60)
    
    batch_count = 0
    
    # 每次获取1年的数据
    while current_start < end:
        current_end = min(current_start + timedelta(days=365*10), end)
        
        start_str = current_start.strftime('%Y%m%d')
        end_str = current_end.strftime('%Y%m%d')
        
        try:
            df = pro.daily(
                ts_code=ts_code,
                start_date=start_str,
                end_date=end_str
            )
            
            if not df.empty:
                all_data.append(df)
                batch_count += 1
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: {len(df):4d} 条")
            else:
                print(f"[{batch_count:2d}] {start_str} ~ {end_str}: 无数据")
            
            time.sleep(sleep_time)  # 避免请求过快
            
        except Exception as e:
            print(f"❌ 获取数据失败 [{start_str} ~ {end_str}]: {e}")
        
        current_start = current_end + timedelta(days=1)
    
    print("=" * 60)
    
    # 合并所有数据
    if all_data:
        result = pd.concat(all_data, ignore_index=True)
        # 去重并排序
        result = result.drop_duplicates(subset=['trade_date']).sort_values('trade_date')
        print(f"✅ 日线数据获取完成，共 {len(result)} 条记录")
        return result
    else:
        print("❌ 未获取到任何数据")
        return pd.DataFrame()

In [26]:
def get_all_daily_basic(ts_code, start_date, sleep_time=0.3):
    """
    一次性获取全部历史 daily_basic 数据（包含流通股本等指标）
    
    Parameters:
    -----------
    ts_code : str
        股票代码，如 '000155.SZ'
    start_date : str
        开始日期，格式 'YYYYMMDD'
    sleep_time : float
        请求间隔时间（秒）
    
    Returns:
    --------
    pd.DataFrame
        包含全部历史 daily_basic 数据的 DataFrame
    """
    end_date = datetime.now().strftime('%Y%m%d')
    
    print(f"开始获取 daily_basic 数据: {start_date} ~ {end_date}")
    print("=" * 60)
    
    # 在两次请求之间等待
    time.sleep(sleep_time)
    
    try:
        df = pro.daily_basic(
            ts_code=ts_code,
            start_date=start_date,
            end_date=end_date,
            # 注意：移除了 close 字段，因为 daily 接口已经提供了
            fields='ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv'
        )
        
        if not df.empty:
            # 去重并排序
            df = df.drop_duplicates(subset=['trade_date']).sort_values('trade_date')
            print(f"✅ daily_basic 数据获取完成，共 {len(df)} 条记录")
            print("=" * 60)
            return df
        else:
            print("❌ 未获取到任何数据")
            print("=" * 60)
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ 获取数据失败: {e}")
        print("=" * 60)
        return pd.DataFrame()

In [27]:
TS_CODE = '000155.SZ'  # 300418.SZ  # 中联重科

# 使用通用的早期日期，无需手动查找上市日期
# 1990-01-01 早于所有A股上市日期，API会自动从实际上市日开始返回数据
START_DATE = '19900101'

In [28]:
# 获取全部历史日线数据
df_daily = get_all_daily_data(TS_CODE, START_DATE, sleep_time=0.3)

# 查看数据
print("\n数据预览:")
display(df_daily.head())
print("\n数据信息:")
print(df_daily.info())

开始获取日线数据: 19900101 ~ 20251102
[ 0] 19900101 ~ 19991230: 无数据
[ 1] 19991231 ~ 20091228: 2113 条
[ 2] 20091229 ~ 20191227: 1911 条
[ 3] 20191228 ~ 20251102: 1394 条
✅ 日线数据获取完成，共 5418 条记录

数据预览:


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
2112,000155.SZ,20000926,8.77,8.90,8.08,8.20,6.18,2.02,32.69,488338.0,405979.0456
2111,000155.SZ,20000927,8.10,8.35,7.91,8.28,8.20,0.08,0.98,142650.0,115943.7322
2110,000155.SZ,20000928,8.27,8.27,8.00,8.02,8.28,-0.26,-3.14,69393.0,56175.6786
2109,000155.SZ,20000929,8.00,8.47,7.98,8.24,8.02,0.22,2.74,77963.0,64525.1906
2108,000155.SZ,20001009,8.28,8.30,8.06,8.23,8.24,-0.01,-0.12,29680.0,24140.1727



数据信息:
<class 'pandas.core.frame.DataFrame'>
Index: 5418 entries, 2112 to 4024
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ts_code     5418 non-null   object 
 1   trade_date  5418 non-null   object 
 2   open        5418 non-null   float64
 3   high        5418 non-null   float64
 4   low         5418 non-null   float64
 5   close       5418 non-null   float64
 6   pre_close   5418 non-null   float64
 7   change      5418 non-null   float64
 8   pct_chg     5418 non-null   float64
 9   vol         5418 non-null   float64
 10  amount      5418 non-null   float64
dtypes: float64(9), object(2)
memory usage: 507.9+ KB
None


## 5. 获取 daily_basic 数据

In [29]:
# 获取全部历史 daily_basic 数据
df_basic = get_all_daily_basic(TS_CODE, START_DATE, sleep_time=0.3)

# 查看数据
print("\n数据预览:")
display(df_basic.head())
print("\n数据信息:")
print(df_basic.info())

开始获取 daily_basic 数据: 19900101 ~ 20251102
✅ daily_basic 数据获取完成，共 5418 条记录

数据预览:


,ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
5417,000155.SZ,20000926,37.5645,37.5645,NaN,38.3116,38.3116,7.3509,3.7316,3.7316,NaN,NaN,47000.0,13000.0,13000.0,385400.0,106600.0
5416,000155.SZ,20000927,10.9731,10.9731,NaN,38.6854,38.6854,7.4226,3.7680,3.7680,NaN,NaN,47000.0,13000.0,13000.0,389160.0,107640.0
5415,000155.SZ,20000928,5.3379,5.3379,NaN,37.4706,37.4706,7.1896,3.6497,3.6497,NaN,NaN,47000.0,13000.0,13000.0,376940.0,104260.0
5414,000155.SZ,20000929,5.9972,5.9972,NaN,38.4985,38.4985,7.3868,3.7498,3.7498,NaN,NaN,47000.0,13000.0,13000.0,387280.0,107120.0
5413,000155.SZ,20001009,2.2831,2.2831,NaN,38.4518,38.4518,7.3778,3.7453,3.7453,NaN,NaN,47000.0,13000.0,13000.0,386810.0,106990.0



数据信息:
<class 'pandas.core.frame.DataFrame'>
Index: 5418 entries, 5417 to 0
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ts_code          5418 non-null   object 
 1   trade_date       5418 non-null   object 
 2   turnover_rate    5418 non-null   float64
 3   turnover_rate_f  5418 non-null   float64
 4   volume_ratio     5413 non-null   float64
 5   pe               4481 non-null   float64
 6   pe_ttm           4004 non-null   float64
 7   pb               5180 non-null   float64
 8   ps               5418 non-null   float64
 9   ps_ttm           5418 non-null   float64
 10  dv_ratio         4404 non-null   float64
 11  dv_ttm           2151 non-null   float64
 12  total_share      5418 non-null   float64
 13  float_share      5418 non-null   float64
 14  free_share       5418 non-null   float64
 15  total_mv         5418 non-null   float64
 16  circ_mv          5418 non-null   float64
dtypes: float64(1

## 6. 合并数据

In [30]:
# 合并日线数据和 basic 数据
df_merged = pd.merge(
    df_daily, 
    df_basic, 
    on=['ts_code', 'trade_date'], 
    how='left',
    suffixes=('', '_basic')
)

print(f"✅ 数据合并完成")
print(f"   - 日线数据: {len(df_daily)} 条")
print(f"   - basic数据: {len(df_basic)} 条")
print(f"   - 合并后: {len(df_merged)} 条")

# 查看合并后的数据
print("\n合并后数据预览:")
display(df_merged.head())

# 查看列名
print("\n列名:")
print(df_merged.columns.tolist())

✅ 数据合并完成
   - 日线数据: 5418 条
   - basic数据: 5418 条
   - 合并后: 5418 条

合并后数据预览:


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,...,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000155.SZ,20000926,8.77,8.90,8.08,8.20,6.18,2.02,32.69,488338.0,...,7.3509,3.7316,3.7316,NaN,NaN,47000.0,13000.0,13000.0,385400.0,106600.0
1,000155.SZ,20000927,8.10,8.35,7.91,8.28,8.20,0.08,0.98,142650.0,...,7.4226,3.7680,3.7680,NaN,NaN,47000.0,13000.0,13000.0,389160.0,107640.0
2,000155.SZ,20000928,8.27,8.27,8.00,8.02,8.28,-0.26,-3.14,69393.0,...,7.1896,3.6497,3.6497,NaN,NaN,47000.0,13000.0,13000.0,376940.0,104260.0
3,000155.SZ,20000929,8.00,8.47,7.98,8.24,8.02,0.22,2.74,77963.0,...,7.3868,3.7498,3.7498,NaN,NaN,47000.0,13000.0,13000.0,387280.0,107120.0
4,000155.SZ,20001009,8.28,8.30,8.06,8.23,8.24,-0.01,-0.12,29680.0,...,7.3778,3.7453,3.7453,NaN,NaN,47000.0,13000.0,13000.0,386810.0,106990.0



列名:
['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close', 'change', 'pct_chg', 'vol', 'amount', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'dv_ratio', 'dv_ttm', 'total_share', 'float_share', 'free_share', 'total_mv', 'circ_mv']


## 7. 数据质量检查

In [31]:
print("="*60)
print("数据质量检查报告")
print("="*60)

# 基本统计
print(f"\n1. 基本信息")
print(f"   股票代码: {TS_CODE}")
print(f"   数据起始: {df_merged['trade_date'].min()}")
print(f"   数据终止: {df_merged['trade_date'].max()}")
print(f"   总记录数: {len(df_merged)}")

# 缺失值检查
print(f"\n2. 缺失值统计")
missing = df_merged.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(missing)
else:
    print("   ✅ 无缺失值")

# 重复值检查
print(f"\n3. 重复值检查")
duplicates = df_merged.duplicated(subset=['trade_date']).sum()
print(f"   重复日期数: {duplicates}")

# 数据范围
print(f"\n4. 价格数据统计")
print(f"   最高价: {df_merged['high'].max():.2f}")
print(f"   最低价: {df_merged['low'].min():.2f}")
print(f"   平均收盘价: {df_merged['close'].mean():.2f}")

# 成交量统计
print(f"\n5. 成交量统计")
print(f"   平均成交量: {df_merged['vol'].mean()/10000:.2f} 万手")
print(f"   最大成交量: {df_merged['vol'].max()/10000:.2f} 万手")

# 股本统计（如果有）
if 'total_share' in df_merged.columns:
    print(f"\n6. 股本统计（最新）")
    latest = df_merged.iloc[-1]
    print(f"   总股本: {latest['total_share']:.2f} 万股")
    if pd.notna(latest.get('float_share')):
        print(f"   流通股本: {latest['float_share']:.2f} 万股")
    if pd.notna(latest.get('free_share')):
        print(f"   自由流通股本: {latest['free_share']:.2f} 万股")

print("\n" + "="*60)

数据质量检查报告

1. 基本信息
   股票代码: 000155.SZ
   数据起始: 20000926
   数据终止: 20251031
   总记录数: 5418

2. 缺失值统计
dv_ttm          3267
pe_ttm          1414
dv_ratio        1014
pe               937
pb               238
volume_ratio       5
dtype: int64

3. 重复值检查
   重复日期数: 0

4. 价格数据统计
   最高价: 39.26
   最低价: 3.09
   平均收盘价: 8.43

5. 成交量统计
   平均成交量: 13.99 万手
   最大成交量: 231.82 万手

6. 股本统计（最新）
   总股本: 184616.83 万股
   流通股本: 184616.83 万股
   自由流通股本: 111765.29 万股



## 8. 保存数据到本地

将数据保存为多种格式，方便后续使用

In [32]:
# 创建 data 目录（如果不存在）
os.makedirs('data', exist_ok=True)

# 文件路径
csv_path = f'data/{TS_CODE}.csv'


# 保存合并数据
print("开始保存数据...")
print("="*60)

# 1. 保存合并后的完整数据
df_merged.to_csv(csv_path, index=False, encoding='utf-8-sig',index_label=True)
print(f"✅ 合并数据已保存: {csv_path}")
print(f"   文件大小: {os.path.getsize(csv_path) / 1024:.2f} KB")


开始保存数据...
✅ 合并数据已保存: data/000155.SZ.csv
   文件大小: 965.45 KB


## 9. 测试读取保存的数据

In [45]:
# 测试读取保存的数据
df1 = pd.read_csv("data/000155.sz.csv")

df2 = pd.read_csv("data/300809.SZ.csv")

print(df1.columns)
print(df2.columns)
print(df1.tail())

Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'close_basic', 'turnover_rate',
       'turnover_rate_f', 'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm',
       'dv_ratio', 'dv_ttm', 'total_share', 'float_share', 'free_share',
       'total_mv', 'circ_mv'],
      dtype='object')
Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'close_basic', 'turnover_rate',
       'turnover_rate_f', 'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm',
       'dv_ratio', 'dv_ttm', 'total_share', 'float_share', 'free_share',
       'total_mv', 'circ_mv'],
      dtype='object')
        ts_code  trade_date   open   high    low  close  pre_close  change  \
5172  000155.SZ    20251024  10.92  11.26  10.84  10.99      10.93    0.06   
5173  000155.SZ    20251027  11.21  11.27  11.00  11.05      10.99    0.06   
5174  000155.SZ    20251028  11.06  11.25  11.

## 10. 数据说明

### 保存的文件

1. **`data/000155_SZ_full_history.csv`** - 完整数据（日线 + basic 合并）
2. **`data/000155_SZ_daily.csv`** - 仅日线数据
3. **`data/000155_SZ_basic.csv`** - 仅 basic 数据

### 主要字段说明

#### 日线数据字段：
- `ts_code`: 股票代码
- `trade_date`: 交易日期
- `open`: 开盘价
- `high`: 最高价
- `low`: 最低价
- `close`: 收盘价
- `pre_close`: 昨收价
- `change`: 涨跌额
- `pct_chg`: 涨跌幅 (%)
- `vol`: 成交量（手）
- `amount`: 成交额（千元）

#### Basic 数据字段：
- `turnover_rate`: 换手率 (%)
- `turnover_rate_f`: 换手率（自由流通股） (%)
- `volume_ratio`: 量比
- `pe`: 市盈率（总市值/净利润）
- `pe_ttm`: 市盈率TTM
- `pb`: 市净率（总市值/净资产）
- `ps`: 市销率
- `ps_ttm`: 市销率TTM
- `dv_ratio`: 股息率 (%)
- `dv_ttm`: 股息率TTM (%)
- `total_share`: 总股本（万股）
- `float_share`: 流通股本（万股）
- `free_share`: 自由流通股本（万股）
- `total_mv`: 总市值（万元）
- `circ_mv`: 流通市值（万元）

### 后续使用

```python
# 读取完整数据
df = pd.read_csv('data/000155_SZ_full_history.csv')

# 转换日期格式
df['trade_date'] = pd.to_datetime(df['trade_date'])

# 按日期排序
df = df.sort_values('trade_date')
```